# Collecting Data

In [ ]:
import cv2
import numpy as np

face_classifier = cv2.CascadeClassifier('../Data/Haarcascades/haarcascade_frontalface_default.xml')

def face_extractor(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 1)
    if faces is ():
        return None
    x, y, w, h = faces[0]
    offset_factor = 50
    return gray[x-offset_factor:x+w+offset_factor, y-offset_factor:y+h+offset_factor]

def generateDataFor(username):
    cap = cv2.VideoCapture(0)
    samples_num = 3000
    count = 0
    while True:
        ret, frame = cap.read()
        face = face_extractor(frame)
        if face is not None:
            cv2.imshow('Original', frame)
            cv2.imshow('Face', face)
            count += 1
            file_name = '../Data/Output/faces/' + username + '/' + str(count) + '.jpg'
            cv2.imwrite(file_name, face)
            if cv2.waitKey(1) == 27 or count == samples_num:
                break
    print("Done Collecting data")
    cap.release()
    cv2.destroyAllWindows()
    
generateDataFor('desha')

# Training Data

In [ ]:
import cv2
import os
import numpy as np

training_data_path = '../Data/Output/faces'
training_labels = [f for f in os.listdir(training_data_path)] 

all_features = []
all_labels = []

for i,label in enumerate(training_labels):
    for f in os.listdir(training_data_path + '/' + label):
        all_features.append(cv2.imread(training_data_path + '/' + label + '/' + f, 0))
        all_labels.append(i)

recognizer = cv2.face.LBPHFaceRecognizer_create()

recognizer.train(all_features, np.array(all_labels))

recognizer.write('../Data/Output/faces_model.yml')

print('Training Done')

# Testing (Terminator Vision Face Recognition App)

In [ ]:
import numpy as np
import cv2
import os

def terminator_vision(img):
    b,g,r = cv2.split(img)
    img = cv2.merge((b*0,g*0,r))
    return img

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('../Data/Output/faces_model.yml')

faceCascade = cv2.CascadeClassifier("../Data/Haarcascades/haarcascade_frontalface_default.xml")

people = [f for f in os.listdir('../Data/Output/faces')] 

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, 1.2,5)
    
    frame = terminator_vision(frame)
    
    for(x,y,w,h) in faces:
        label, confidence = recognizer.predict(gray[y:y+h,x:x+w])
        cv2.rectangle(frame, (x-20,y-20), (x+w+20,y+h+20), (0,255,0), 4)
        cv2.rectangle(frame, (x-22,y-90), (x+w+22, y-22), (0,255,0), -1)
        cv2.putText(frame, str(people[label]), (x,y-40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 3)
    cv2.imshow('Terminator Vision Face Recognition App', frame) 
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()